In [1]:
print('hello')

hello


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from huggingface_hub import notebook_login
import torch
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

import os
import sys


/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
document=[]
for file in os.listdir("./trizPDF/"):
  if file.endswith(".pdf"):
    pdf_path="./trizPDF/"+file
    loader=PyPDFLoader(pdf_path)
    document.extend(loader.load())
  elif file.endswith('.docx') or file.endswith('.doc'):
    doc_path="./trizPDF/"+file
    loader=Docx2txtLoader(doc_path)
    document.extend(loader.load())
  elif file.endswith('.txt'):
    text_path="./trizPDF/"+file
    loader=TextLoader(text_path)
    document.extend(loader.load())

In [7]:
document

[Document(page_content=" \n \nCopyright © Oxford Creativity     www.triz.co.uk   +44(0) 1993 882 461       1 \n \n40 Inventive Principles With Examples  \n \nPrinciple 1 Segmentation  \nA. Divide an object into independent parts  \n\uf02d\uf020Different focal length lenses for a camera  \n\uf02d\uf020Gator -grip socket spanner  \n\uf02d\uf020Multi -pin connectors  \n\uf02d\uf020Multiple pistons in an internal combustion engine  \n\uf02d\uf020Multi -engined aircraft  \n\uf02d\uf020Stratification of different constituen ts inside a chemical process vessel  \n \nB. Make an object sectional - easy to assemble or disassemble  \n\uf02d\uf020Rapid -release fasteners for bicycle saddle/wheel/etc  \n\uf02d\uf020Quick disconnect joints in plumbing and hydraulic systems  \n\uf02d\uf020Single fastener V -band clamps on flange j oints  \n\uf02d\uf020Loose -leaf paper in a ring binder  \n \nC. Increase the degree of fragmentation or segmentation  \n\uf02d\uf020Multiple control surfaces on aerodynami

In [8]:
len(document)

12

In [9]:
document_splitter=CharacterTextSplitter(separator='\n', chunk_size=500, chunk_overlap=100)

In [10]:
document_chunks=document_splitter.split_documents(document)
len(document_chunks)

88

In [11]:
document_chunks[0]

Document(page_content='Copyright © Oxford Creativity     www.triz.co.uk   +44(0) 1993 882 461       1 \n \n40 Inventive Principles With Examples  \n \nPrinciple 1 Segmentation  \nA. Divide an object into independent parts  \n\uf02d\uf020Different focal length lenses for a camera  \n\uf02d\uf020Gator -grip socket spanner  \n\uf02d\uf020Multi -pin connectors  \n\uf02d\uf020Multiple pistons in an internal combustion engine  \n\uf02d\uf020Multi -engined aircraft  \n\uf02d\uf020Stratification of different constituen ts inside a chemical process vessel', metadata={'source': './trizPDF/triz_40_inventive_principles.pdf', 'page': 0})

In [12]:
#Embedding model load

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

model_name = "BAAI/bge-large-en-v1.5"
# model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    # ,
    # query_instruction="为这个句子生成表示以用于检索相关文章："
)


emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())

[codecarbon WARNING @ 15:23:45] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 15:23:45] [setup] RAM Tracking...
[codecarbon INFO @ 15:23:45] [setup] GPU Tracking...
[codecarbon INFO @ 15:23:45] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:23:45] [setup] CPU Tracking...
[codecarbon WARNING @ 15:23:45] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1716301425.1712587


[codecarbon WARNING @ 15:23:46] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 15:23:46] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 15:23:46] >>> Tracker's metadata:
[codecarbon INFO @ 15:23:46]   Platform system: Linux-5.4.0-174-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 15:23:46]   Python version: 3.12.1
[codecarbon INFO @ 15:23:46]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 15:23:46]   Available RAM : 503.339 GB
[codecarbon INFO @ 15:23:46]   CPU count: 112
[codecarbon INFO @ 15:23:46]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 15:23:46]   GPU count: 2
[codecarbon INFO @ 15:23:46]   GPU model: 2 x NVIDIA A40
[codecarbon INFO @ 15:23:51] Energy consumed for RAM : 0.000109 kWh. RAM Power : 188.75217247009277 W
[codecarbon INFO @ 15:23:51] Energy consumed for all GPUs : 0.000059 kWh. Total GPU Power : 101.946347950

Emissions for this code execution is:  5.0390431266635e-05
End of code time:  1716301431.7418413


In [13]:
print(model)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-large-en-v1.5' cache_folder=None model_kwargs={} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: ' embed_instruction=''


In [14]:
#Chroma DB saving files and embeddings

from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

chroma_db = Chroma.from_documents(
    document_chunks, model, persist_directory="./db_triz"
)

chroma_db.persist()

emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())


[codecarbon WARNING @ 15:39:38] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 15:39:38] [setup] RAM Tracking...
[codecarbon INFO @ 15:39:38] [setup] GPU Tracking...
[codecarbon INFO @ 15:39:38] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:39:38] [setup] CPU Tracking...
[codecarbon WARNING @ 15:39:38] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1716302378.1785464


[codecarbon WARNING @ 15:39:39] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 15:39:39] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 15:39:39] >>> Tracker's metadata:
[codecarbon INFO @ 15:39:39]   Platform system: Linux-5.4.0-174-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 15:39:39]   Python version: 3.12.1
[codecarbon INFO @ 15:39:39]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 15:39:39]   Available RAM : 503.339 GB
[codecarbon INFO @ 15:39:39]   CPU count: 112
[codecarbon INFO @ 15:39:39]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 15:39:39]   GPU count: 2
[codecarbon INFO @ 15:39:39]   GPU model: 2 x NVIDIA A40
[codecarbon INFO @ 15:39:44] Energy consumed for RAM : 0.000072 kWh. RAM Power : 188.75217247009277 W
[codecarbon INFO @ 15:39:44] Energy consumed for all GPUs : 0.000065 kWh. Total GPU Power : 168.714878775

Emissions for this code execution is:  4.005470442937089e-05
End of code time:  1716302384.2088256


In [15]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig


In [17]:
model_id = "/home/c7361293/triz_llm/llama-2-7b-chat/model"
 
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
                 model_id,
                 device_map="auto",
                 use_auth_token=True,
                #  load_in_8bit=True
                 )



/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.52s/it]


In [18]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map='auto',
              max_new_tokens=512,
              min_new_tokens=-1,
              top_k=30
              )

In [19]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [20]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f2ace1e0080>, model_kwargs={'temperature': 0})

In [21]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [23]:
#Create our Q/A Chain
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                             retriever=chroma_db.as_retriever(search_kwargs={'k':6}),
                                             verbose=False, memory=memory)

In [24]:
result=pdf_qa({"question":"""
The present invention aims to improve the vertically-integrated nonvolatile memory device by incorporating a peripheral circuit structure with a peripheral circuit therein, and a cell array structure bonded to the peripheral circuit structure. The cell array structure includes a cell area and a connection area, where the cell area has a staircase-shaped cell stack and a plurality of capacitor core contact structures. Each capacitor core contact structure consists of a first core conductor electrically connected to the peripheral circuit, and a first cover insulating layer extending between the first core conductor and the plurality of gate electrodes. The invention improves the memory device's performance by increasing the capacitance of the capacitor structures, which in turn enhances the device's energy efficiency. However, the invention may face challenges in manufacturing and scaling due to the complexity of the structure.
The paragraph provided matches which principles and why. Can you give me in points and one line for the reason.
"""})

/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `30` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [26]:
result['answer']

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nCopyright © Oxford Creativity     www.triz.co.uk   +44(0) 1993 882 461       6 \n \nPrinciple 17 Another Dimension  \nA. Move into an additional dimension - from one to two - from two to three  \n\uf02d\uf020Coiled telephone wire  \n\uf02d\uf020Curved bristles on a brush  \n\uf02d\uf020Pizza -box with ribbed (as opposed to flat) base  \n\uf02d\uf020Spiral staircase uses less floor area  \n\uf02d\uf020Introduction of down and up slopes between stations on railway reduces  overall power requirements\n\nB. Go from single storey or layer to multi -storey or multi -layered  \n\uf02d\uf020Player with many CDs  \n\uf02d\uf020Stacked or multi -layered circuit boards  \n\uf02d\uf020Multi -storey office blocks or car -parks  \n \nC. Incline an object, lay it on its side  \n\uf02d\uf020Cars on road transporter inclined to save space

In [35]:
text_input = result["answer"]
summary_index = text_input.find("Helpful Answer:\n")
summary = text_input
if summary_index != -1:
    # Extract the text from "Begin Summary:" to the end of the string
        summary = text_input[summary_index:]

print(summary.strip())

Helpful Answer:

Points:

1. Principle 17 - Another Dimension
Reason: The staircase-shaped cell stack in the cell array structure allows for a reduction in overall power requirements by using less floor area.

2. Principle 33 - Homogeneity
Reason: The peripheral circuit structure and the cell array structure are made of the same material (or material with identical properties), which reduces chemical reactions and improves the device's performance.

3. Principle 7 - Nested Doll
Reason: The cell array structure is placed inside the peripheral circuit structure, which allows for the reduction of the need to raise or lower the cell array structure.

4. Principle 12 - Equipotentiality
Reason: The connection area of the cell array structure is bonded to the peripheral circuit structure, which eliminates the need to raise or lower the cell array structure.


In [ ]:
# import re

# def extract_helpful_answer(text):
#     # Define a more flexible regular expression pattern to find the "Helpful Answer" section
#     # This pattern will account for variations in whitespace and the presence of different types of list items
#     pattern = r"Helpful Answer:\s*((?:•\s*.+?(?:\n|$))+)"

#     # Use re.search to find the pattern in the text
#     match = re.search(pattern, text, re.DOTALL)

#     # Check if a match is found
#     if match:
#         # Return the matched text, stripping any leading/trailing whitespace for cleanliness
#         return match.group()
#     else:
#         # Return a message if no match is found
#         return "No 'Helpful Answer' section found."

# # Example text input
# text_input = result["answer"]

# # Call the function with the example text
# extracted_answer = extract_helpful_answer(text_input)
# print(extracted_answer)